# 로컬에서 훈련 하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [1]:
from dotenv import load_dotenv

import os

HF_TOKEN = os.getenv('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [2]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [3]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 

print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)

data_folder:  ../data/naver-news-summarization-ko
train_data_json:  ../data/naver-news-summarization-ko/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test_dataset.json


In [4]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. 베이스 모델 준비

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B"
output_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news"

### Config YAML 파일 생성

In [6]:
%%writefile local_llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id:  "meta-llama/Meta-Llama-3-8B" # Hugging Face model id
dataset_path: "../data/naver-news-summarization-ko"                      # path to dataset
max_seq_len:  2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

Overwriting local_llama_3_8b_fsdp_qlora.yaml


## 3. 훈련 Script 실행

In [7]:
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 \
../scripts/local_run_fsdp_qlora.py \
--config local_llama_3_8b_fsdp_qlora.yaml

[2024-06-29 10:15:44,226] torch.distributed.run: [WARNING] 
[2024-06-29 10:15:44,226] torch.distributed.run: [WARNING] *****************************************
[2024-06-29 10:15:44,226] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-06-29 10:15:44,226] torch.distributed.run: [WARNING] *****************************************
/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use

## 4. 베이스 모델과 훈련된 모델 머지

In [8]:
model_id, output_dir

('meta-llama/Meta-Llama-3-8B',
 '/home/ec2-user/SageMaker/models/llama-3-8b-naver-news')

### 모델 머지 및 로컬에 저장

In [9]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.96it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 머지된 모델 로딩

In [10]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.32s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 5. 추론

### 테스트 데이터 셋 로딩

### 추론

In [16]:
from datasets import load_dataset 
from random import randint

# Load our test dataset
eval_dataset = load_dataset("json", data_files=test_data_json, split="train")

# Test on sample 
rand_idx = randint(0, len(eval_dataset))
messages = eval_dataset[rand_idx]["messages"][:2]

input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id= tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]

print(f"**Query:**\n{eval_dataset[rand_idx]['messages'][1]['content']}\n")
print(f"**Original Answer:**\n{eval_dataset[rand_idx]['messages'][2]['content']}\n")
print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

코스피지수가 개인과 기관의 매수세에 상승 출발했다. 일러스트 정다운 1일 오전 9시 4분 기준 코스피지수는 전날보다 19.85포인트 0.85% 상승한 2352.49를 기록 중이다. 유가증권시장에서 개인과 기관이 각각 60억원 706억원을 사들이며 지수를 끌어올리고 있다. 외국인은 770억원 어치를 매물로 출회중이다. 같은 시각 코스닥지수는 전날 대비 7.04포인트 0.94% 오른 752.48을 기록하고 있다. 코스닥시장에서는 개인과 외국인이 각각 93억원 75억원을 사들이는 가운데 기관 투자자는 68억원을 팔아치우고 있다. 간밤 미국 뉴욕 증시에서 주요 3대 지수는 경기 침체 우려가 지속되는 가운데 일제히 하락했다. 지난 6월 30일 현지 시각 미 뉴욕증권거래소 NYSE 에서 다우존스30산업평균지수는 전날보다 0.82% 하락했다. 스탠더드앤드푸어스 S P 500지수와 나스닥지수도 각각 0.88% 1.33% 떨어졌다.

**Original Answer:**
코스피지수가 개인과 기관의 매수세에 상승 출발했으며 1일 오전 9시 4분 기준 코스피지수는 전날보다 19.85포인트 0.85% 상승한 2352.49를 기록 중이고 미국 뉴욕 증시에서 주요 3대 지수는 경기 침체 우려가 지속되는 가운데 일제히 하락했다.

**Generated Answer:**
1일 오전 9시 4분 기준 코스피지수는 전날보다 19.85포인트 0.85% 상승한 2352.49를 기록 중이다. 개인과 기관이 각각 60억원 706억원을 사들이며 지수를 끌어올리고 있다. 외국인은 770억원 어치를 매물로 출회중이다. 같은 시각 코스닥지수는 전날 대비 7.04포인트 0.94% 오른 752.48을 기록하고 있다. 코스닥시장에서는 개인과 외국인이 각각 93억원 75억원을 사들이는 가운데 기관 투자자는 68억원을 팔아치우고 있다. 간밤 미국 뉴욕 증시에서 주요 3대 지수는 경기 침체 우려가 